# Comparación de precios de iPhones en Perú

Se utilizarán tecnicas de web scraping para conseguir información sobre las distintas tiendas.

Coders:
- Anchante Fernandez, Marcello
- Batalla Flores, Stephano

# Objetivos

- Determinar precios de 3 tiendas conocidas: Saga Falabella, Hiraoka y Mercado Libre.

## Resultados de iPhone's vendidos por Falabella

In [ ]:
!pip install bs4
!pip install requests
!pip install pandas

In [9]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

In [4]:
def obtener_enlaces(url):
    content = requests.get(url).text
    soup = BeautifulSoup(content, 'html.parser')
    enlaces_similares = re.findall(r"https://www\.falabella\.com\.pe/falabella-pe/product/\w+", content)
    celulares_enlaces = list(set(enlaces_similares))
        
    return celulares_enlaces
    
    

In [5]:
celulares_enlaces = obtener_enlaces('https://www.falabella.com.pe/falabella-pe/category/cat760706/Celulares-y-Telefonos?sortBy=derived.price.search%2Cdesc&facetSelected=true&f.derived.variant.sellerId=FALABELLA&f.product.brandName=apple')

In [11]:
def precio(url):
 
    precios = []
    titulos = []
    marcas = []
    modelos = []
    capacidades = []
    ext = []

    for u in url:
        content = requests.get(u).text
        soup = BeautifulSoup(content, 'html.parser')
        
        titulo = soup.find('div', class_='jsx-1442607798')
        precio = soup.find('span', class_='copy17')

        if titulo and precio:
            titulos.append(titulo.text)
            precios.append(int(precio.text.replace(' ', '').replace(',', '').replace('S/','')))

            marca = 'Apple'
            modelo_match = re.search(r'iPhone\s([\w\s]+)\s\d+GB', titulo.text)
            modelo = modelo_match.group(1).strip() if modelo_match else ""
            capacidad_match = re.search(r'\d+GB', titulo.text)
            capacidad = capacidad_match.group(0) if capacidad_match else ""
            extras_match = re.search(r'\+\s*(.*)$', titulo.text)
            extras = extras_match.group(1).strip() if extras_match else "-"
            
            capacidad = re.sub(r'(\d+)(GB)', r'\1 \2', capacidad)

            marcas.append(marca)
            modelos.append('iPhone ' + modelo.lstrip().rstrip())
            capacidades.append(capacidad)
            ext.append(extras)
        
    tabla = pd.DataFrame({'Marca': marcas, 
                          'Modelo': modelos, 
                          'Capacidad': capacidades, 
                          'Extras': ext, 
                          'Precio (S/.)': precios, 
                          'Enlace':(celulares_enlaces),
                          'Tienda':'Falabella'})
    return tabla


In [12]:
falabella = precio(celulares_enlaces)
falabella

,Marca,Modelo,Capacidad,Extras,Precio (S/.),Enlace,Tienda
0,Apple,iPhone SE,64 GB,-,1499,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
1,Apple,iPhone 14,128 GB,Audífonos AirPods 2nda Generación,5849,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
2,Apple,iPhone 14,128 GB,Audífonos AirPods 2nda Generación,5849,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
3,Apple,iPhone 14,128 GB,-,4399,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
4,Apple,iPhone 14 Pro,128 GB,-,5499,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
5,Apple,iPhone 14 Pro,256 GB,-,5999,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
6,Apple,iPhone 13,128 GB,-,5099,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
7,Apple,iPhone 13,256 GB,-,3799,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
8,Apple,iPhone 14,256 GB,-,4899,https://www.falabella.com.pe/falabella-pe/prod...,Falabella
9,Apple,iPhone 11,128 GB,Case iPhone 11 Transparente,3599,https://www.falabella.com.pe/falabella-pe/prod...,Falabella


## Resultados de iPhone's vendidos por MercadoLibre

In [13]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

In [14]:
def tabla_mercadolibre(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    product_list = soup.find_all("div", class_="ui-search-result__content-wrapper")
    resultados = soup.find_all('span', attrs={#'class':"andes-money-amount__fraction",
                                              #'aria-hidden':'true'
                                              'class':'andes-money-amount ui-search-price__part shops__price-part andes-money-amount--cents-superscript'})
    
    

    precios = []
    for resultado in resultados:
        precios.append(int(resultado.find_all('span', class_='andes-money-amount__fraction')[0].text.replace('.','')))
    

    mar = []
    mod = []
    cap = []
    col = []
        

    for product in product_list:

        name = product.find("h2", class_="ui-search-item__title").text.strip()
        
        marca = 'Apple'

        modelo = re.findall(r"(.*?)\s\(", name)
        if len(modelo) > 0:
            modelo = re.findall(r"(.*?)\s\(", name)[0]
        else: modelo = '-'

        capacidad = re.findall(r"\((.*?)\)",name)[0]
        color = re.findall(r"\-\s(.+)$",name)
        if len(color) > 0:
            color = re.findall(r"\-\s(.+)$", name)[0]
        else: color = 'No especificado'
        
        capacidad = re.sub(r'(\d+)\s*([gG]b)', r'\1 GB', capacidad)
        mar.append(marca)
        mod.append(modelo.replace('Apple ',''))
        cap.append(capacidad)
        col.append(color)
       
    # return mar, mod, cap, col, precios
    return pd.DataFrame({'Marca':mar,
                            'Modelo':mod,
                            'Capacidad':cap,
                            'Color':col,
                            'Precio (S/.)':precios,
                            'Tienda':'Mercado Libre'})

In [15]:
mercado_libre =(tabla_mercadolibre('https://listado.mercadolibre.com.pe/celulares-telefonos/celulares-smartphones/apple/nuevo/_Tienda_apple_af_to#unapplied_filter_id%3Dinstallments%26unapplied_filter_name%3DPago%26unapplied_value_id%3Dno_interest%26unapplied_value_name%3DSin+inter%C3%A9s%26unapplied_autoselect%3Dfalse'))
mercado_libre

,Marca,Modelo,Capacidad,Color,Precio (S/.),Tienda
0,Apple,iPhone 13,128 GB,Azul medianoche,3349,Mercado Libre
1,Apple,iPhone 13,128 GB,Blanco estelar,3349,Mercado Libre
2,Apple,iPhone 11,128 GB,Negro,2349,Mercado Libre
3,Apple,iPhone 14 Pro Max,256 GB,Morado oscuro,6299,Mercado Libre
4,Apple,iPhone 14 Pro Max,128 GB,Negro espacial,5699,Mercado Libre
5,Apple,iPhone 13,128 GB,Azul,3349,Mercado Libre
6,Apple,iPhone 11,128 GB,Blanco,2349,Mercado Libre
7,Apple,iPhone 14 Pro Max,256 GB,Negro espacial,6299,Mercado Libre
8,Apple,iPhone 14 Pro,128 GB,Morado oscuro,5299,Mercado Libre
9,Apple,iPhone 14 Pro Max,128 GB,Color oro,5699,Mercado Libre
